In [1]:
 #需导入的模块
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
import matplotlib.pyplot as plt
import datetime
from numpy import random #经常有用
import re
import pymongo

#将pymongo数据转换成Dataframe
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["db_lianjia"]
mycol = mydb["chengjiao"]

x = list(mycol.find())
df=DataFrame(x)
#把没用的id删除掉
df=df.drop('_id',axis=1)  
str_datetime=lambda x:datetime.datetime.strptime(x,'%Y-%m-%d')
df['成交日期']=df['成交日期'].apply(str_datetime)

KeyError: '成交日期'

In [ ]:
#获得每日成交量
df['count']=0
d=df.groupby(['成交日期']).count()['count']

import matplotlib.pyplot as plt
plt.rcParams['font.family']=['SIMHEI']
plt.figure(figsize=(20, 2))
plt.plot(d.index,d.values,label='每日成交量',linewidth=1,marker='o')
plt.xlabel('日期',fontsize=20)
plt.ylabel('套数',fontsize=20)
plt.xticks(d.index,list(map(lambda x:x.date().strftime('%m-%d'),d.index)))
plt.legend()
for x,y in zip(d.index,d.values):
    plt.text(x, y+0.05, y, ha='center', va= 'bottom',fontsize=15)
plt.show()
 

In [ ]:
#获得某个单价区间的成交量
bins=[5000,10000,15000,20000,25000,30000]
cats=pd.cut(df['单价'],bins)
X=np.arange(1,6)
Y=pd.value_counts(cats).values
x_label=list(map(lambda x:str(x/10000)+'-'+str((x+5000)/10000),bins[:5]))
color=['red','black','peru','orchid','deepskyblue']
plt.rcParams['font.family']=['SIMHEI']
plt.bar(x=X,height=Y,align='center',width=0.5,label='不同单价的销售量',color='g')
plt.xticks(X,x_label)
plt.xlabel('单价（万）',fontsize=20)
plt.ylabel('销售量',fontsize=20)
plt.legend(loc='best')
for x,y in zip(X,Y):
    s=str(y)+'套'
    plt.text(x, y+0.05, s, ha='center', va= 'bottom',fontsize=15)
plt.show()

In [ ]:
#获得某个小区的成交信息
try:
    df=df.drop(columns=['count'])
    dwj=df[df['小区'].apply(lambda x:'第五郡' in x and '公寓' not in x)]
    t=dwj.sort_values(['单价'])
    t.describe().round(0).unstack()
except KeyError as e:
    pass
dwj.sort_values(by=['成交日期'])